Autor:
Ramon Fernandes Viana

#Implementação de um sistema BCI-SSVEP
---



Este script faz parte da implementação das etapas de construção de uma BCI (Brain Computer Interface).

Este trabalho contém as seguintes partes: Entrada de dados, Pré-Processamento, Estração de Características, Seleção de Características e Classificação



A base de dados utilizada neste trabalho pode ser acessada pelo link:
http://bci.med.tsinghua.edu.cn/download.html

In [ ]:
#Bibliotecas científicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sp
import scipy.io as sio
from scipy import signal
from scipy.fft import fft
import math 
from numpy import array
from scipy.io import loadmat


#Biblioteca de aprendizado de máquina
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from sklearn.cross_decomposition import CCA
from tensorflow.keras.optimizers import *

##Importando a base de dados e selecionando os indivíduos

In [ ]:
#Biblioteca para acessar o Google Drive
from google.colab import drive

#Endereço da base de dados no Google Drive
path_database = '/content/gdrive/MyDrive/Base de dados IC/S34.mat'
# O sinal filtrado (CAR) para o indivíduo 34 está em:
# /content/gdrive/MyDrive/Colab Notebooks/Iniciação/Sinais_Filtrados/CAR/filtragem_CAR.mat

#Autoriza a leitura do Google Drive
drive.mount('/content/gdrive')

# O indivíduo testado -- 34          
mat = sio.loadmat(path_database, verify_compressed_data_integrity=False) 
data = np.array(mat['data'])
data.shape

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


(64, 1500, 40, 6)

In [ ]:
# Parâmetros de entrada
# conforme definido no artigo, o sinal coletado tem duração de 6s no entanto os 0,5s iniciais e finais são desconsiderados
int_dados = data[:,125:1375,:,:]
rows,cols,freqs,trials = int_dados.shape
freqList = [0,2,4,7]
freqReal = [8,10,12,15]
numFreq = len(freqList)
# Frequência de Amostragem
Fs = 250;
res_esp = Fs/1250;
freq = np.arange(1250)*(res_esp);
# vetor de tempo
time = np.arange(1250)*1/Fs;

##Encontrando as localizações de cada frequência

In [ ]:
freq_new = np.arange(0,250,1)
freq_new.shape
def frequencias(freq_new,numFreq, freqReal):
  binFreq = [0 for i in range(numFreq)]
  for localizacao in range(numFreq):  
    binFreq[localizacao] = np.where(freq_new == freqReal[localizacao]) 
  return binFreq  
posicoesFreq = frequencias(freq_new,numFreq, freqReal)
posicoesFreq = np.asarray(posicoesFreq)
for i in range(numFreq):
  posicoesFreq[i] = posicoesFreq[i].item() 
posicoesFreq  

array([[[ 8]],

       [[10]],

       [[12]],

       [[15]]])

##Extração de Caracterísitcas - Análise com a FFT

In [ ]:
# matriz H - Extração com Método da FFT
H_FFT = np.zeros((janelas*numFreq*trials,numFreq*rows))
H_AUX = np.zeros(numFreq)
# número de janelas do sinal
janelas = 5                  
for freqs_sinal in range(numFreq):
  for trial in range(trials):
    for jan in range(janelas):
      for canal in range (rows):
        janela_inicio = jan*250
        janela_fim = (jan+1)*250
        janela = np.arange(janela_inicio,janela_fim)
        FFT =  np.abs(fft(int_dados[canal,janela,freqList[freqs_sinal],trial]))
        for indices in range(numFreq):
          H_AUX[indices] = FFT[posicoesFreq[indices]]
        H_FFT [trials*janelas*freqs_sinal+jan+(janelas*(trial)), canal*numFreq: ((canal+1)*numFreq)] = H_AUX
    

## Seleção de Atributos através do método de correlação de Pearson

In [ ]:
# SELECOA DE ATRIBUTOS COM PEARSON
#*************************************************
A = np.array(H_FFT);  #matriz de características
uns_pos = np.ones((janelas*trials,rows))
uns_neg = -np.ones((janelas*trials,rows))

Vet_1 = np.array(np.vstack((uns_pos,uns_neg,uns_pos,uns_neg)))
Vet_2 = np.array(np.vstack((uns_neg,uns_pos,uns_neg,uns_pos)))

B = np.array(np.hstack((Vet_1,Vet_2,Vet_1,Vet_2)));
B.shape

(120, 256)

##Calculos dos eletrodos mais significativos para as frequências desejadas

In [ ]:
Vetor_R =  np.zeros((A.shape[1],B.shape[1]))

for freq in range (B.shape[1]):
  for ch in range (A.shape[1]):
    print(freq,ch)
    print(A[:,ch], B[:,freq])
    R = np.corrcoef(A[:,ch],B[:,freq])
    Vetor_R[ch,freq] = np.abs(R[1,0])

print(Vetor_R.shape)
indices_canais = np.argsort(Vetor_R, axis=0)[::-1]
print(indices_canais.shape)

A saída de streaming foi truncada nas últimas 5000 linhas.
 113.75180765 210.52146624 327.67624229 192.38157138 119.07406566
 170.17955535 693.60363482 215.37087808  92.51957472 257.07333387
 112.20194969 437.98254714 222.03936567  79.16001011 247.78372905
 277.32295844 155.72458193 273.7118788  530.98347272 289.32485707
 355.94160568 304.53499461 389.88940415 219.68011299 269.83428129
 237.22130789  27.46742411 117.50985812 105.98307607 207.45137415
 225.81693575 172.43630015 157.22104636  64.38826384 335.2217686
 285.06532647 449.64727784 421.72130033 346.59521982 302.46909344
  91.42346675 195.80036481  47.02987475 257.30257236 102.7686209
  53.12314138 176.40000109 191.46867873 209.30734051 122.1804363
 160.17415303 186.28069722 283.87496247 345.90184353 295.86174957] [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

# Classificador Linear

## Matriz de rótulos

In [ ]:
np.ones((num_rotulos,num_rotulos)).shape
Ri = -np.ones((num_rotulos,num_rotulos,num_rotulos,num_rotulos)) + 2*np.identity(num_rotulos)
Ri.ravel()
Ri.shape

(30, 30, 30, 30)

In [ ]:
Rt1.shape

(120, 1)

In [ ]:
num_rotulos = trials*janelas
R1 = np.ones((num_rotulos,1))
R2 = -np.ones((num_rotulos,1))
Rt1 = np.vstack((R1,R2,R2,R2))
Rt2 = np.vstack((R2,R1,R2,R2))
Rt3 = np.vstack((R2,R2,R1,R2))
Rt4 = np.vstack((R2,R2,R2,R1))
R = np.hstack((Rt1,Rt2,Rt3,Rt4))
print(R.shape)

uns = np.ones((len(H_FFT),1))
# matriz atributos
H = np.array(np.hstack((H_FFT,uns)));
H.shape


(120, 4)


(120, 257)

Classificação

SVM para todos os eletrodos


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

#Considerando as matrizes com os dados em H e a matriz de rótulos R - equivalente ao código inicial do Matlab
#Particionar os dados usando 20% para validacao
Ht, Hv, Rt, Rv = train_test_split(H, R, test_size = 0.20)
#SVM - treinando
modelo1 = svm.SVC(kernel='linear').fit(Ht, Rt[:,0])
modelo2 = svm.SVC(kernel='linear').fit(Ht, Rt[:,1])
modelo3 = svm.SVC(kernel='linear').fit(Ht, Rt[:,2])
modelo4 = svm.SVC(kernel='linear').fit(Ht, Rt[:,3])
#SVM - validando
Y1 = modelo1.predict(Hv)
Y2 = modelo2.predict(Hv)
Y3 = modelo3.predict(Hv)
Y4 = modelo4.predict(Hv)
Y = np.vstack([Y1,Y2,Y3,Y4]).transpose()
# Y = Y1.transpose
index = np.argmax(Y, axis=1)
index2 = np.argmax(Rv, axis=1)
# print(index)
# print(index2)
# contando os acertos
corretos = 0

for i in range(len(Rv)):
    #if((Y[i,:]==Rv[i,:])):
    if(index[i] == index2[i]):
      corretos = corretos + 1

val_acc = (corretos/len(Rv))*100
print('Acurácia da validação:', val_acc)


Acurácia da validação: 91.66666666666666
